In [1]:
import pandas as pd 
import numpy as np
import sqlite3
conn = sqlite3.connect('CourseData2.db')
c = conn.cursor()

In [2]:
%load_ext sql

In [3]:
%%sql
sqlite:///CourseData2.db

'Connected: @CourseData2.db'

In [4]:
%%sql
--Dropping tables for when we need to restart and clear CourseData database-- 
--Drop ERD tables--
DROP TABLE IF EXISTS SECTION;
DROP TABLE IF EXISTS CATALOG;
DROP TABLE IF EXISTS INSTRUCTOR;
DROP TABLE IF EXISTS COURSE_MEETING;

--Drop import tables--
DROP TABLE IF EXISTS import_course;
DROP TABLE IF EXISTS import_course_meeting;
DROP TABLE IF EXISTS import_course_catalog;

 * sqlite:///CourseData2.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Create Tables (Import and ERD) ##

In [5]:
%%sql
--Creating import_catalog table-- 

CREATE TABLE 'import_course_catalog' (
    program_code TEXT NOT NULL,
    program_name VARCHAR NOT NULL,
    catalog_id VARCHAR NOT NULL,
    course_title VARCHAR NOT NULL, 
    credits VARCHAR NOT NULL,
    prereqs VARCHAR, 
    coreqs VARCHAR,
    fees VARCHAR,
    attributes VARCHAR,
    description VARCHAR 
);

 * sqlite:///CourseData2.db
Done.


[]

In [6]:
%%sql
--Creating import_course_meeting table-- 

CREATE TABLE 'import_course_meeting' (
    term VARCHAR NOT NULL,
    crn INTEGER NOT NULL,
    location VARCHAR NOT NULL,
    day TEXT NOT NULL,
    start VARCHAR NOT NULL,
    end VARCHAR NOT NULL 
);

 * sqlite:///CourseData2.db
Done.


[]

In [7]:
%%sql
--Creating import_course table-- 

CREATE TABLE 'import_course' (
    term VARCHAR NOT NULL,
    crn INTEGER NOT NULL,
    catalog_id VARCHAR NOT NULL,
    section VARCHAR NOT NULL,
    credits VARCHAR,
    title VARCHAR, 
    meetings VARCHAR,
    timecodes VARCHAR, 
    primary_instructor TEXT,
    cap INTEGER, 
    act INTEGER, 
    rem INTEGER 
);

 * sqlite:///CourseData2.db
Done.


[]

In [8]:
%%sql
-- Creating table INSTRUCTOR with surrogate primary key-- 

CREATE TABLE 'INSTRUCTOR' (
    InID INTEGER NOT NULL PRIMARY KEY,
    Name TEXT NOT NULL
);

--Creating table CATALOG with CatID as the primary key-- 

CREATE TABLE 'CATALOG'(
    CatalogClassifier INTEGER NOT NULL PRIMARY KEY,
    CatID VARCHAR,
    Title VARCHAR NOT NULL,
    Description TEXT,
    Credits TEXT NOT NULL,
    PName TEXT NOT NULL,
    PCode TEXT,
    PREREQS TEXT,
    COREQS TEXT,
    FEES INTEGER,
    Attribute TEXT
    
);

--Creating table SECTION with surrogate primary key-- 

CREATE TABLE 'SECTION'(
    SID INTEGER NOT NULL PRIMARY KEY,
    CRN INTEGER NOT NULL,
    Term TEXT NOT NULL,
    Section VARCHAR NOT NULL,
    Cap INTEGER,
    Act INTEGER,
    Rem INTEGER,
    CatalogClassifier INTEGER,
    InID INTEGER,
    FOREIGN KEY (CatalogClassifier) REFERENCES CATALOG(CatalogClassifier),
    FOREIGN KEY (InID) REFERENCES INSTRUCTOR(InID)
);

--Creating table COURSE_MEETING with CatalogClassifier as the Primary Key-- 

CREATE TABLE 'COURSE_MEETING' (
    MID INTEGER NOT NULL PRIMARY KEY,
    CRN INTEGER NOT NULL,
    Term TEXT NOT NULL,
    Start TEXT NOT NULL,
    Location VARCHAR NOT NULL,
    Day TEXT NOT NULL,
    End TEXT NOT NULL,
    FOREIGN KEY (CRN) REFERENCES SECTION(CRN),
    FOREIGN KEY (Term) REFERENCES SECTION(Term)
);

 * sqlite:///CourseData2.db
Done.
Done.
Done.
Done.


[]

## Populate the Import Tables with the Data from the csv files ##

**Creating dataframes for the import course**

In [9]:
## populating import_course table
fall = ['SourceData/Fall2014/courses.csv','SourceData/Fall2015/courses.csv', 'SourceData/Fall2016/courses.csv',
        'SourceData/Fall2017/courses.csv','SourceData/Fall2018/courses.csv']
dff0 = pd.read_csv(fall[0])
dff1 = pd.read_csv(fall[1])
dff2 = pd.read_csv(fall[2])    
dff3 = pd.read_csv(fall[3])
dff4 = pd.read_csv(fall[4])

In [10]:
df_fall = pd.concat([dff0, dff1, dff2, dff3, dff4], axis=0)
df_fall.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
90,Fall2014,72985,AH 0164,A,3.000,American Art: Civil War to Civil Rights,"[{'days': 'W', 'times': '0200pm-0430pm', 'date...",['W 0200pm-0430pm 09/02-12/08 CNS 6'],Philip I. Eliasoph,25,24,1
1023,Fall2014,71540,NS 0272C,E,0.000,Geriatric Nursing Clinical,"[{'days': 'F', 'times': '0700am-0200pm', 'date...",['F 0700am-0200pm 09/05-10/10 JHE '],Katherine Marie Saracino,7,6,1
1281,Fall2018,74745,PY 0212,01,4.000,Developmental Psychology for Majors with Lab,"[{'days': 'TF', 'times': '0930am-1215pm', 'dat...",['TF 0930am-1215pm 09/04-12/10 BNW 124'],Michael Walden Creane,0,19,-19
1263,Fall2014,72952,PY 0111,B,3.000,Developmental Psychology for Non-Majors,"[{'days': 'MR', 'times': '0800am-0915am', 'dat...",['MR 0800am-0915am 09/02-12/08 BNW 254A'],Barbara Lynn Welles,30,30,0
371,Fall2017,71729,ED 0401,01,1.000,K-12 Teaching Internship Seminar for Initital ...,"[{'days': 'R', 'times': '0400pm-0450pm', 'date...","['R 0400pm-0450pm 09/07-09/07 CNS 204', 'R 040...",Hildegard H. Szokol,0,15,-15


In [11]:
df_fall.describe()

,crn,cap,act,rem
count,7486.000000,7486.00000,7486.000000,7486.000000
mean,74523.769169,18.39487,16.310446,2.219076
std,2512.545835,14.41373,9.863494,17.175757
min,70001.000000,0.00000,-1.000000,-75.000000
25%,72213.000000,12.00000,8.000000,0.000000
50%,74848.500000,19.00000,16.000000,1.000000
75%,76698.750000,25.00000,24.000000,5.000000
max,79215.000000,500.00000,90.000000,999.000000


In [12]:
spring = ['SourceData/Spring2015/courses.csv','SourceData/Spring2016/courses.csv', 'SourceData/Spring2017/courses.csv',
        'SourceData/Spring2018/courses.csv','SourceData/Spring2019/courses.csv', 'SourceData/SpringBreak2017/courses.csv']
dfs0 = pd.read_csv(spring[0])
dfs1 = pd.read_csv(spring[1])
dfs2 = pd.read_csv(spring[2])    
dfs3 = pd.read_csv(spring[3])
dfs4 = pd.read_csv(spring[4])
dfs5 = pd.read_csv(spring[5])

In [13]:
df_spring = pd.concat([dfs0,dfs1,dfs2,dfs3,dfs4,dfs5], axis=0)
df_spring.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
452,Spring2018,34254,EN 0012,39,3.000,Texts and Contexts II: Writing About Literature,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dat...",['TF 1230pm-0145pm 01/16-05/01 DMH 131'],Frank Moliterno,19,18,1
931,Spring2015,32016,MU 0300,A,3.000,Independent Study,[],[],Laura Nash,0,0,0
587,Spring2018,34313,FR 0111,02,3.000,Elementary French II,"[{'days': 'TF', 'times': '0930am-1020am', 'dat...","['TF 0930am-1020am 01/16-05/01 CNS 204', 'W 09...",Zoe Erotopoulos,18,18,0
295,Spring2017,33025,CO 0561,QZ,3.000,Thesis Research,[],[],Qin Zhang,0,1,-1
1145,Spring2019,37482,PH 0215,01,3.000,Philosophy of Science,"[{'days': 'MW', 'times': '0500pm-0615pm', 'dat...",['MW 0500pm-0615pm 01/22-04/30 CNS 304'],J. Brooks Colburn,20,20,0


In [14]:
df_spring.describe()

,crn,cap,act,rem
count,7122.000000,7122.000000,7122.000000,7122.000000
mean,35212.001966,18.123982,15.658804,2.465178
std,2618.670948,10.876640,10.342885,6.497100
min,31001.000000,0.000000,0.000000,-47.000000
25%,32773.250000,11.000000,7.000000,0.000000
50%,34835.000000,19.000000,16.000000,1.000000
75%,37693.750000,25.000000,24.000000,4.000000
max,41702.000000,100.000000,90.000000,89.000000


In [15]:
summer = ['SourceData/Summer2015/courses.csv', 'SourceData/Summer2016/courses.csv',
        'SourceData/Summer2017/courses.csv','SourceData/Summer2018/courses.csv']
dfsm0 = pd.read_csv(summer[0])
dfsm1 = pd.read_csv(summer[1])
dfsm2 = pd.read_csv(summer[2])    
dfsm3 = pd.read_csv(summer[3])

In [16]:
df_summer = pd.concat([dfsm0,dfsm1,dfsm2,dfsm3], axis=0)
df_summer.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
47,Summer2018,58345,CH 0111L,01,1.000,General Chemistry I Lab,"[{'days': 'TWR', 'times': '1230pm-0330pm', 'da...",['TWR 1230pm-0330pm 06/04-07/06 BNW 361'],Jon J. Harper,16,7,9
203,Summer2018,58073,NS 0312C,01,0.000,Medical Surgical Nursing I Clinical,"[{'days': 'W', 'times': '0700am-1100am', 'date...","['W 0700am-1100am 05/23-05/23 VA ', 'WR 0700am...",Laura A. Conklin,8,8,0
119,Summer2018,58198,IS 0391,01,3.000,Internship,[],[],Sarah Jennie Bollinger,0,0,0
247,Summer2015,55031,PH 0101,A,3.000,Introduction to Philosophy,"[{'days': 'MTWRF', 'times': '0930am-1230pm', '...",['MTWRF 0930am-1230pm 07/06-07/17 DMH 231'],Kris F. Sealey,0,9,-9
54,Summer2016,56042,CN 0447,01,3.000,Lifespan Human Development,"[{'days': 'MW', 'times': '0100pm-0900pm', 'dat...","['MW 0100pm-0900pm 05/23-05/25 CNS 204', 'MW 0...",Virginia A. Kelly,7,7,0


In [17]:
df_summer.describe()

,crn,cap,act,rem
count,1224.000000,1224.000000,1224.000000,1224.000000
mean,56704.305556,14.843954,10.913399,3.928922
std,1143.898206,10.771969,9.124774,6.881239
min,55001.000000,0.000000,-1.000000,-24.000000
25%,55382.750000,6.000000,5.000000,0.000000
50%,56517.500000,16.000000,9.000000,3.000000
75%,58008.500000,20.000000,15.000000,9.000000
max,58477.000000,60.000000,59.000000,24.000000


In [18]:
winter = ['SourceData/Winter2015/courses.csv', 'SourceData/Winter2016/courses.csv',
        'SourceData/Winter2017/courses.csv','SourceData/Winter2018/courses.csv']
dfw0 = pd.read_csv(winter[0])
dfw1 = pd.read_csv(winter[1])
dfw2 = pd.read_csv(winter[2])    
dfw3 = pd.read_csv(winter[3])

In [19]:
df_winter = pd.concat([dfw0,dfw1,dfw2,dfw3], axis=0)
df_winter.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
23,Winter2016,11611,PY 0121,OL1,3.000,Fundamentals of Social Psychology,[],[],Dorothea D. Braginsky,25,21,4
14,Winter2016,11630,GS 0399,MW,3.000,General Studies Senior Project,[],[],Margaret A. Wills,0,1,-1
8,Winter2016,11606,EC 0011,OL1,3.000,Introduction to Microeconomics,[],[],William F. Vasquez Mazariegos,25,12,13
18,Winter2015,10903,MU 0102,A,3.000,History and Development of Rock,"[{'days': 'MTW', 'times': '0830am-0430pm', 'da...",['MTW 0830am-0430pm 01/05-01/13 JGS 25'],Brian Q. Torff,15,15,0
24,Winter2015,10905,SA 0139,A,3.000,Watercolor,"[{'days': 'MTWRF', 'times': '0830am-0430pm', '...",['MTWRF 0830am-0430pm 01/05-01/09 LYL 8'],Suzanne Chamlin-Richer,15,18,-3


In [20]:
df_winter.describe()

,crn,cap,act,rem
count,105.000000,105.000000,105.000000,105.000000
mean,11524.266667,17.133333,12.342857,4.790476
std,355.382575,7.612069,6.584990,6.207463
min,10893.000000,0.000000,0.000000,-3.000000
25%,11603.000000,15.000000,9.000000,0.000000
50%,11633.000000,20.000000,12.000000,4.000000
75%,11802.000000,20.000000,18.000000,9.000000
max,11841.000000,30.000000,25.000000,27.000000


In [21]:
df_course = pd.concat([df_fall, df_spring, df_summer, df_winter], axis=0)
df_course.sample(5)

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
954,Fall2018,74142,MA 0217,03,3.000,Accelerated Statistics,"[{'days': 'TF', 'times': '1100am-1215pm', 'dat...",['TF 1100am-1215pm 09/04-12/10 CNS 301'],Shabnam Hashemiyeh,28,29,-1
982,Spring2016,37828,NS 0314C,E,0.000,Maternal and Newborn Nursing Clinical,"[{'days': 'W', 'times': '0700am-0200pm', 'date...",['W 0700am-0200pm 03/16-05/03 STM '],Anka Roberto,7,4,3
1069,Fall2015,75635,NS 0323C,C,0.000,Pediatric Nursing Clinical,"[{'days': 'W', 'times': '0700am-0100pm', 'date...",['W 0700am-0100pm 09/01-12/07 YAL '],Maura Byrnes-Casey,6,6,0
483,Fall2014,73219,EN 0012,D,3.000,Texts and Contexts II: Writing About Literature,"[{'days': 'MR', 'times': '0330pm-0445pm', 'dat...",['MR 0330pm-0445pm 09/02-12/08 CNS 203'],Lisa M. Breunig,19,20,-1
272,Summer2016,56401,RLD 0592,01,3.000,Practicum in Structured Literacy Interventions,"[{'days': 'TR', 'times': '0830am-1130am', 'dat...","['TR 0830am-1130am 07/05-07/07 CNS 10', 'TWR 0...",Jule J. McCombes-Tolis,10,9,1


In [22]:
df_course.describe()

,crn,cap,act,rem
count,15937.000000,15937.000000,15937.000000,15937.000000
mean,55172.299178,17.992784,15.578591,2.477317
std,19308.879968,12.671906,10.107690,12.709781
min,10893.000000,0.000000,-1.000000,-75.000000
25%,35426.000000,10.000000,7.000000,0.000000
50%,57145.000000,19.000000,15.000000,1.000000
75%,74541.000000,25.000000,23.000000,5.000000
max,79215.000000,500.000000,90.000000,999.000000


In [23]:
df_course.to_sql('import_course', conn, if_exists='append', index=False)

In [24]:
%%sql
SELECT *
FROM import_course
LIMIT 10;

 * sqlite:///CourseData2.db
Done.


term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
Fall2014,70384,AC 0011,C01,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31
Fall2014,70385,AC 0011,C02,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0930am-1045am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0930am-1045am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31
Fall2014,70382,AC 0011,C03,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 1230pm-0145pm 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31
Fall2014,70291,AC 0011,C04,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1100am-1215pm', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 1100am-1215pm 09/02-12/08 DSB 111'],Rebecca I. Bloch,0,29,-29
Fall2014,70350,AC 0011,C05,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 1230pm-0145pm 09/02-12/08 DSB 111'],Rebecca I. Bloch,0,30,-30
Fall2014,70381,AC 0011,C06,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '0330pm-0445pm', 'dates': '09/02-12/08', 'location': 'DSB 110A'}]",['MR 0330pm-0445pm 09/02-12/08 DSB 110A'],Rebecca I. Bloch,0,31,-31
Fall2014,70383,AC 0011,G,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '0930am-1045am', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 0930am-1045am 09/02-12/08 DSB 111'],Paul Caster,30,31,-1
Fall2014,70391,AC 0011,H,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0930am-1045am', 'dates': '09/02-12/08', 'location': 'DSB 110A'}]",['TF 0930am-1045am 09/02-12/08 DSB 110A'],Jo Ann Drusbosky,30,32,-2
Fall2014,71105,AC 0011,I,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '1100am-1215pm', 'dates': '09/02-12/08', 'location': 'DSB 110A'}]",['TF 1100am-1215pm 09/02-12/08 DSB 110A'],Jo Ann Drusbosky,30,33,-3
Fall2014,71123,AC 0011,J,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0200pm-0315pm', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0200pm-0315pm 09/02-12/08 DSB 105'],Jo Ann Drusbosky,30,32,-2


**Creating dataframes for the import course_meeting**

In [25]:
cm_fall = ['SourceData/Fall2014/course_meetings.csv', 'SourceData/Fall2015/course_meetings.csv',
           'SourceData/Fall2016/course_meetings.csv', 'SourceData/Fall2017/course_meetings.csv',
          'SourceData/Fall2018/course_meetings.csv']
dfcmf1 = pd.read_csv(cm_fall[0])
dfcmf2  = pd.read_csv(cm_fall[1])
dfcmf3  = pd.read_csv(cm_fall[2])
dfcmf4  = pd.read_csv(cm_fall[3])
dfcmf5  = pd.read_csv(cm_fall[4])

In [26]:
df_cmfall = pd.concat([dfcmf1, dfcmf2, dfcmf3, dfcmf4, dfcmf5], axis=0)
df_cmfall.sample(5)

,term,crn,location,day,start,end
21554,Fall2018,74030,BNW 138,F,2018-10-05T08:00:00,2018-10-05T08:50:00
14678,Fall2016,78835,DSB 107,W,2016-10-26T11:00:00,2016-10-26T13:30:00
3525,Fall2017,71378,DSB 109,W,2017-09-27T08:00:00,2017-09-27T10:30:00
31133,Fall2018,74993,CNS 101,R,2018-10-25T08:00:00,2018-10-25T08:50:00
6871,Fall2016,78941,BNW 253,F,2016-10-28T12:30:00,2016-10-28T13:45:00


In [27]:
cm_spring = ['SourceData/Spring2015/course_meetings.csv',
           'SourceData/Spring2016/course_meetings.csv', 'SourceData/Spring2017/course_meetings.csv',
          'SourceData/Spring2018/course_meetings.csv', 'SourceData/Spring2019/course_meetings.csv']
dfcms1 = pd.read_csv(cm_spring[0])
dfcms2  = pd.read_csv(cm_spring[1])
dfcms3  = pd.read_csv(cm_spring[2])
dfcms4  = pd.read_csv(cm_spring[3])
dfcms5  = pd.read_csv(cm_spring[4])


In [28]:
df_cmspring = pd.concat([dfcms1, dfcms2, dfcms3, dfcms4, dfcms5], axis=0)
df_cmspring.sample(5)

,term,crn,location,day,start,end
5722,Spring2018,34841,CNS 304,W,2018-01-17T11:00:00,2018-01-17T13:30:00
26883,Spring2015,33901,DMH 231,M,2015-03-02T15:30:00,2015-03-02T16:20:00
1392,Spring2016,38820,CNS 306,F,2016-04-01T11:00:00,2016-04-01T12:15:00
28025,Spring2015,34019,MCA 107,R,2015-02-05T11:00:00,2015-02-05T11:50:00
6466,Spring2017,31337,CNS 101,F,2017-02-17T14:00:00,2017-02-17T15:15:00


In [29]:
cm_springbreak = ['SourceData/SpringBreak2017/course_meetings.csv']

dfcmsb1 = pd.read_csv(cm_springbreak[0])


In [30]:
cm_summer = ['SourceData/Summer2015/course_meetings.csv',
           'SourceData/Summer2016/course_meetings.csv', 'SourceData/Summer2017/course_meetings.csv',
          'SourceData/Summer2018/course_meetings.csv']
dfcmsu1 = pd.read_csv(cm_summer[0])
dfcmsu2  = pd.read_csv(cm_summer[1])
dfcmsu3  = pd.read_csv(cm_summer[2])
dfcmsu4  = pd.read_csv(cm_summer[3])

In [31]:
df_cmsummer = pd.concat([dfcmsu1, dfcmsu2, dfcmsu3, dfcmsu4], axis=0)
df_cmspring.sample(5)

,term,crn,location,day,start,end
15790,Spring2016,37243,DSB 111,T,2016-03-15T12:30:00,2016-03-15T13:45:00
5291,Spring2016,37188,CNS 104,F,2016-04-08T14:00:00,2016-04-08T15:15:00
26333,Spring2017,32721,CNS 305,T,2017-01-24T14:00:00,2017-01-24T15:15:00
18496,Spring2017,33006,BNW GR22,T,2017-01-31T17:00:00,2017-01-31T18:15:00
13710,Spring2015,33244,CNS 209,W,2015-02-04T09:00:00,2015-02-04T09:50:00


In [32]:
cm_winter = ['SourceData/Winter2015/course_meetings.csv',
           'SourceData/Winter2016/course_meetings.csv', 'SourceData/Winter2017/course_meetings.csv',
          'SourceData/Winter2018/course_meetings.csv']
dfcmw1 = pd.read_csv(cm_winter[0])
dfcmw2  = pd.read_csv(cm_winter[1])
dfcmw3  = pd.read_csv(cm_winter[2])
dfcmw4  = pd.read_csv(cm_winter[3])

In [33]:
df_cmwinter = pd.concat([dfcmw1, dfcmw2, dfcmw3, dfcmw4], axis=0)
df_cmwinter.sample(5)

,term,crn,location,day,start,end
139,Winter2017,11721,CNS 101,S,2017-01-07T09:00:00,2017-01-07T15:00:00
7,Winter2016,11614,DSB 109,R,2016-01-07T08:30:00,2016-01-07T16:30:00
9,Winter2015,10913,DSB 115,F,2015-01-16T10:00:00,2015-01-16T18:00:00
49,Winter2015,10912,DSB 107,S,2015-01-17T08:30:00,2015-01-17T16:30:00
42,Winter2018,11834,DSB 106,S,2018-01-06T09:00:00,2018-01-06T17:30:00


In [34]:
df_coursemeeting = pd.concat([df_cmfall,df_cmspring,dfcmsb1,df_cmsummer,df_cmwinter],axis=0)

In [35]:
df_coursemeeting.to_sql('import_course_meeting', conn, if_exists='append', index=False)

In [36]:
%%sql
SELECT *
FROM import_course_meeting
LIMIT 10;

 * sqlite:///CourseData2.db
Done.


term,crn,location,day,start,end
Fall2014,70384,DSB 105,T,2014-09-02T08:00:00,2014-09-02T09:15:00
Fall2014,70384,DSB 105,F,2014-09-05T08:00:00,2014-09-05T09:15:00
Fall2014,70384,DSB 105,T,2014-09-09T08:00:00,2014-09-09T09:15:00
Fall2014,70384,DSB 105,F,2014-09-12T08:00:00,2014-09-12T09:15:00
Fall2014,70384,DSB 105,T,2014-09-16T08:00:00,2014-09-16T09:15:00
Fall2014,70384,DSB 105,F,2014-09-19T08:00:00,2014-09-19T09:15:00
Fall2014,70384,DSB 105,T,2014-09-23T08:00:00,2014-09-23T09:15:00
Fall2014,70384,DSB 105,F,2014-09-26T08:00:00,2014-09-26T09:15:00
Fall2014,70384,DSB 105,T,2014-09-30T08:00:00,2014-09-30T09:15:00
Fall2014,70384,DSB 105,F,2014-10-03T08:00:00,2014-10-03T09:15:00


**Creating the dataframes for import_course_catalog**

In [37]:
cc2017_2018 = ['SourceData/Catalogs/CourseCatalog2017_2018.csv']
dfcc17_18 = pd.read_csv(cc2017_2018[0])
dfcc17_18.sample(5)

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
1676,ED,Education,ED 0459,Developmental Reading in the Secondary School,3 Credits,NaN,NaN,NaN,NaN,This course emphasizes enhancing reading compr...
638,BI,Biology,BI 0262L,Human Physiology Lab,0 Credits,NaN,NaN,NaN,NaN,NaN
318,SA,Studio Art,SA 0014,Introduction to Printmaking,3 Credits,NaN,NaN,$55 Materials Fee,"GDSA Graphic Design: Studio Arts, VPC2 Visual ...","This course is an introduction to traditional,..."
442,SL,TESOL and Bilingual Education,SL 0526,Historical and Sociopolitical Issues in Biling...,3 Credits,NaN,NaN,NaN,NaN,"This course, which is conducted as a seminar, ..."
1387,MD,Educational Technology,MD 0406,Introduction to Reference,3 Credits,NaN,NaN,NaN,NaN,Candidates will explore print and online resou...


In [38]:
cc2018_2019 = ['SourceData/Catalogs/CourseCatalog2018_2019.csv']
dfcc18_19 = pd.read_csv(cc2018_2019[0])
dfcc18_19.sample(5)

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
1711,CI,Chinese,CI 0252,The City and Modern China,3 Credits,NaN,NaN,NaN,"E_AF English Literature After 1800, WDIV World...",The course studies the literary and visual rep...
147,BI,Biology,BI 0015,Fundamentals of Biology I,3 Credits,NaN,NaN,NaN,NaN,"This course, an introductory study of biology ..."
205,TA,Theatre,TA 0030,Acting I,3 Credits,NaN,NaN,NaN,VPC2 Visual and Performing Arts Core: Applied ...,This class is an intensive introduction to tec...
1636,MG,Management,MG 0540,Cross-Cultural Management,3 Credits,NaN,NaN,NaN,NaN,This course develops a framework for distingui...
571,SP,Spanish,SP 0331,Love and Deception in 19th-Century Spanish Lit...,3 Credits,"SP 0245, SP 0251.",NaN,NaN,NaN,Students study and analyze representative work...


In [39]:
df_catalog = pd.concat([dfcc17_18,dfcc18_19], axis=0)
df_catalog.sample(5)

,program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
1748,MA,Mathematics,MA 0418,Applied Statistics II,3 Credits,MA 0417.,NaN,NaN,NaN,This course is a continuation of
76,AH,Art History,AH 0014,Art of Asia,3 Credits,NaN,NaN,NaN,"GDAH Graphic Design: Art History, VPCH Visual ...",This course introduces major monuments of the ...
595,PH,Philosophy,PH 0206,Philosophical Perspectives on Women in Classic...,3 Credits,PH 0101.,NaN,NaN,"WSGF Women, Gender, and Sexuality Studies: Gen...",Ancient Greek and Latin literature presented i...
184,IL,International Studies,IL 0295,Seminar in International Studies,3 Credits,NaN,NaN,NaN,NaN,The course examines special topics in internat...
345,BI,Biology,BI 0170P,General Biology I PLG,0 Credits,NaN,NaN,NaN,NaN,NaN


In [40]:
df_catalog.to_sql('import_course_catalog', conn, if_exists='append', index=False)

In [41]:
%%sql
SELECT *
FROM import_course_catalog
LIMIT 10;

 * sqlite:///CourseData2.db
Done.


program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
AN,Asian Studies,AN 0301,Independent Study,1-3 Credits,None,None,None,None,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.
AN,Asian Studies,AN 0310,Asian Studies Seminar,3 Credits,None,None,None,None,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only."
BU,Business,BU 0211,Legal Environment of Business,3 Credits,Junior standing.,None,None,None,"This course examines the broad philosophical as well as practical nature and function of the legal system, and introduces students to the legal and social responsibilities of business. The course includes an introduction to the legal system, the federal courts, Constitutional law, the United States Supreme Court, the civil process, and regulatory areas such as employment discrimination, protection of the environment, and corporate governance and securities markets."
BU,Business,BU 0220,Environmental Law and Policy,3 Credits,None,None,None,"EVME Environmental Studies Major Elective, EVPE Environmental Studies Elective, EVSS Environmental Studies: Social Science, MGEL Management: General Elective","This course surveys issues arising out of federal laws designed to protect the environment and manage resources. It considers in detail the role of the Environmental Protection Agency in the enforcement of environmental policies arising out of such laws as the National Environmental Policy Act, the Clean Water Act, and the Clear Air Act, among others. The course also considers the impact of Congress, political parties, bureaucracy, and interest groups in shaping environmental policy, giving special attention to the impact of environmental regulation on business and private property rights."
BU,Business,BU 0311,"The Law of Contracts, Sales, and Property",3 Credits,BU 0211.,None,None,None,"This course examines the components of common law contracts including the concepts of offer and acceptance, consideration, capacity and legality, assignment of rights and delegation of duties, as well as discharge of contracts. The course covers Articles 2 and 2A of the Uniform Commercial Code relating to leases, sales of goods, and warranties. The course also considers personal and real property, and bailments."
BU,Business,BU 0312,The Law of Business Organizations and Financial Transactions,3 Credits,BU 0211.,None,None,None,"This course offers an analysis of legal principles related to the law of agency, sole proprietorships, partnerships, corporations, limited liability companies, and other business forms. The second half of the course addresses several sections of the Uniform Commercial Code, such as negotiable instruments, bank collections and deposits and secured transactions. Finally, the course examines the law of suretyship, debtor-creditor relationships, and bankruptcy."
BU,Business,BU 0320,Employment Law and Discrimination in the Workplace,3 Credits,None,None,None,"MGEL Management: General Elective, UDIV U.S. Diversity","This course examines a variety of legal issues related to the workplace including the doctrine of employment at will, employee privacy, and the history and development of labor unions and the legal protections afforded by the National Labor Relations Act. A study of the

## Populate ERD Tables with data from Import Tables ##

In [42]:
%%sql

--Populating Course_Meeting table using import_course_meeting data-- 

INSERT INTO Course_Meeting (CRN, Term, Start, Location, Day, End)
SELECT DISTINCT crn, term, start, location, day, end 
FROM import_course_meeting;

 * sqlite:///CourseData2.db
Done.


[]

In [43]:
%%sql 

--Checking data population of Course_Meeting table-- 

SELECT * 
FROM Course_Meeting
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


MID,CRN,Term,Start,Location,Day,End
1,70384,Fall2014,2014-09-02T08:00:00,DSB 105,T,2014-09-02T09:15:00
2,70384,Fall2014,2014-09-05T08:00:00,DSB 105,F,2014-09-05T09:15:00
3,70384,Fall2014,2014-09-09T08:00:00,DSB 105,T,2014-09-09T09:15:00
4,70384,Fall2014,2014-09-12T08:00:00,DSB 105,F,2014-09-12T09:15:00
5,70384,Fall2014,2014-09-16T08:00:00,DSB 105,T,2014-09-16T09:15:00


In [44]:
%%sql 

--Populating Catalog table using import_course_catalog data--  

INSERT INTO CATALOG (CatID, Title, Description, Credits, PName, PCode, PREREQS, COREQS, FEES, Attribute)
SELECT DISTINCT catalog_id, course_title, description, credits, program_name, program_code, prereqs, coreqs, fees, attributes
FROM import_course_catalog;

 * sqlite:///CourseData2.db
Done.


[]

In [45]:
%%sql 

--Checking data population of Catalog table-- 

SELECT * 
FROM CATALOG
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


CatalogClassifier,CatID,Title,Description,Credits,PName,PCode,PREREQS,COREQS,FEES,Attribute
1,AN 0301,Independent Study,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.,1-3 Credits,Asian Studies,AN,None,None,None,None
2,AN 0310,Asian Studies Seminar,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only.",3 Credits,Asian Studies,AN,None,None,None,None
3,BU 0211,Legal Environment of Business,"This course examines the broad philosophical as well as practical nature and function of the legal system, and introduces students to the legal and social responsibilities of business. The course includes an introduction to the legal system, the federal courts, Constitutional law, the United States Supreme Court, the civil process, and regulatory areas such as employment discrimination, protection of the environment, and corporate governance and securities markets.",3 Credits,Business,BU,Junior standing.,None,None,None
4,BU 0220,Environmental Law and Policy,"This course surveys issues arising out of federal laws designed to protect the environment and manage resources. It considers in detail the role of the Environmental Protection Agency in the enforcement of environmental policies arising out of such laws as the National Environmental Policy Act, the Clean Water Act, and the Clear Air Act, among others. The course also considers the impact of Congress, political parties, bureaucracy, and interest groups in shaping environmental policy, giving special attention to the impact of environmental regulation on business and private property rights.",3 Credits,Business,BU,None,None,None,"EVME Environmental Studies Major Elective, EVPE Environmental Studies Elective, EVSS Environmental Studies: Social Science, MGEL Management: General Elective"
5,BU 0311,"The Law of Contracts, Sales, and Property","This course examines the components of common law contracts including the concepts of offer and acceptance, consideration, capacity and legality, assignment of rights and delegation of duties, as well as discharge of contracts. The course covers Articles 2 and 2A of the Uniform Commercial Code relating to leases, sales of goods, and warranties. The course also considers personal and real property, and bailments.",3 Credits,Business,BU,BU 0211.,None,None,None


In [46]:
%%sql 

--Populating Instructor table using import_courses data--  

INSERT INTO Instructor (Name)
SELECT DISTINCT primary_instructor
FROM import_course
WHERE primary_instructor <> 'TBA' AND primary_instructor NOT LIKE 'primary_instructor' AND primary_instructor NOT LIKE '%/';

 * sqlite:///CourseData2.db
Done.


[]

In [47]:
%%sql 

--Checking data population of Instructor table-- 

SELECT * 
FROM Instructor
LIMIT 5;

 * sqlite:///CourseData2.db
Done.


InID,Name
1,Michael P. Coyne
2,Rebecca I. Bloch
3,Paul Caster
4,Jo Ann Drusbosky
5,Arleen N. Kardos


In [51]:
%%sql 

--Populating the SECTION table using import_course data--
--Joining Instructor table using import_course and instructor table--

INSERT INTO SECTION (CRN, Term, Section, Cap, Act, Rem, CatalogClassifier, InID)
SELECT crn, term, section, cap, act, rem, catalog_id, InID
FROM import_course
    JOIN Instructor ON (import_course.primary_instructor = Instructor.Name)
;

 * sqlite:///CourseData2.db
Done.


[]

In [52]:
%%sql 

--Checking data population of Section table--

SELECT * 
FROM Section
LIMIT 10;

 * sqlite:///CourseData2.db
Done.


SID,CRN,Term,Section,Cap,Act,Rem,CatalogClassifier,InID
1,70384,Fall2014,C01,0,31,-31,AC 0011,1
2,70385,Fall2014,C02,0,31,-31,AC 0011,1
3,70382,Fall2014,C03,0,31,-31,AC 0011,1
4,70291,Fall2014,C04,0,29,-29,AC 0011,2
5,70350,Fall2014,C05,0,30,-30,AC 0011,2
6,70381,Fall2014,C06,0,31,-31,AC 0011,2
7,70383,Fall2014,G,30,31,-1,AC 0011,3
8,70391,Fall2014,H,30,32,-2,AC 0011,4
9,71105,Fall2014,I,30,33,-3,AC 0011,4
10,71123,Fall2014,J,30,32,-2,AC 0011,4


## Drop Import Tables ##

In [53]:
%%sql
--Drop import tables--
DROP TABLE IF EXISTS import_course;
DROP TABLE IF EXISTS import_course_meeting;
DROP TABLE IF EXISTS import_course_catalog;

 * sqlite:///CourseData2.db
Done.
Done.
Done.


[]